In [1]:
# AOI = 'POLYGON((-85.299088 40.339368, -85.332047 40.241477, -85.134979 40.229427, -85.157639 40.34146, -85.299088 40.339368))'
# START_DATE = "2020-07-01"
# END_DATE = "2020-08-01"
# SENTINEL2_GOOGLE_API_KEY = "./.secret/sentinel2_google_api_key.json"
# SATELLITE_CACHE_FOLDER = './img'

# # Output folder
# OUTPUT_FOLDER = './prediction'

In [2]:
from sentinel2download.downloader import Sentinel2Downloader
from sentinel2download.overlap import Sentinel2Overlap
import json
import argparse
import os
import numpy as np

from util import log as ulog
from architectures import ARCH_MAP
from data_generator import DataGenerator
from util.normalization import set_normalization
from util.save_prediction_masks import save_masks_contrast

from util.raster_mosaic import get_img_entry_id, image_grid_overlap
from util.rasterio_dep import proj_rasterio
import pathlib
from PIL import Image, ImageOps, ImageFile
from PIL.PngImagePlugin import PngInfo
import subprocess
import rasterio
from version import __version__, min_cm_vsm_version
from pkg_resources import parse_version
import tensorflow as tf
import urllib.request
import shutil
import subprocess

from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling

import shapely.geometry
import shapely.wkt as wkt
import geopandas
import geojson
import tempfile
import geopandas as gp

In [ ]:
AOI = os.getenv('AOI')
START_DATE = os.getenv("START_DATE")
END_DATE = os.getenv("END_DATE")
SENTINEL2_GOOGLE_API_KEY = os.getenv('SENTINEL2_GOOGLE_API_KEY')
SATELLITE_CACHE_FOLDER = os.getenv('SENTINEL2_CACHE')

# Output folder
OUTPUT_FOLDER = os.getenv('OUTPUT_FOLDER')

In [3]:
PRODUCT_TYPE = 'L1C'  # or L1C
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
BANDS = {'AOT', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'WVP'}
CONSTRAINTS = {'NODATA_PIXEL_PERCENTAGE': 10.0, 'CLOUDY_PIXEL_PERCENTAGE': 5.0, }
MAX_SHIFT_ITERS = 2
MAX_SHIFT = 30

In [4]:
shapely_polygon = wkt.loads(AOI)

In [5]:
aoi = gp.GeoDataFrame(geometry=[wkt.loads(AOI)], crs="epsg:4326")
aoi_filename = "provided_aoi.geojson"
aoi.to_file(aoi_filename, driver="GeoJSON")

In [6]:
def epsg_code(longitude, latitude):
        """
        Generates EPSG code from lon, lat
        :param longitude: float
        :param latitude: float
        :return: int, EPSG code
        """

        def _zone_number(lat, lon):
            if 56 <= lat < 64 and 3 <= lon < 12:
                return 32
            if 72 <= lat <= 84 and lon >= 0:
                if lon < 9:
                    return 31
                elif lon < 21:
                    return 33
                elif lon < 33:
                    return 35
                elif lon < 42:
                    return 37

            return int((lon + 180) / 6) + 1

        zone = _zone_number(latitude, longitude)

        if latitude > 0:
            return 32600 + zone
        else:
            return 32700 + zone

In [7]:
s2overlap = Sentinel2Overlap(aoi_path=aoi_filename)
overlap_tiles = s2overlap.overlap()


In [8]:
def shift_date(date, delta=5, format='%Y-%m-%d'):
    date = datetime.strptime(date, format)
    date = date - timedelta(days=delta)    
    return datetime.strftime(date, format)

In [9]:
def load_images(tiles, start_date, end_date):
    loader = Sentinel2Downloader(SENTINEL2_GOOGLE_API_KEY)
    
    loadings = dict()
        
    for tile in tiles:
        start = start_date
        end = end_date
        
        print(f"Loading images for tile: {tile}...")
        count = 0
        while count < MAX_SHIFT_ITERS:
            loaded = loader.download(PRODUCT_TYPE,
                                [tile],
                                start_date=start,
                                end_date=end,
                                output_dir=SATELLITE_CACHE_FOLDER,                       
                                bands=BANDS,
                                constraints=CONSTRAINTS,
                                full_download=False)
        
            if not loaded:
                end = start_date
                start = shift_date(start_date, delta=MAX_SHIFT) 
                print(f"For tile: {tile} and dates {start_date} {end_date} proper images not found! Shift dates to {start} {end}!")
            else:
                break
            count += 1
        if loaded:
            loadings[tile] = loaded
            print(f"Loading images for tile {tile} finished")
        else:
            print(f"Images for tile {tile} were not loaded!")
    folders = os.listdir(SATELLITE_CACHE_FOLDER)
    for i, folder in enumerate(folders):
        folders[i] += '.SAFE'
        
    loaded = loader.download(PRODUCT_TYPE,
                                [tile],
                                start_date=start,
                                end_date=end,
                                output_dir=SATELLITE_CACHE_FOLDER,                       
                                bands=BANDS,
                                constraints=CONSTRAINTS,
                                full_download=True)
    for fold in os.listdir(SATELLITE_CACHE_FOLDER):
        if fold not in folders:
            shutil.rmtree(os.path.join(SATELLITE_CACHE_FOLDER, fold))
    return loadings

In [10]:
loadings = load_images(overlap_tiles, START_DATE, END_DATE)

Loading images for tile: 16TFK...
Loading images for tile 16TFK finished


In [22]:
def preproc_data(output_dir, subfolder):
    fol = os.path.join(output_dir, subfolder)
    args = f"./cm-vsm/vsm/build/bin/cm_vsm -d {fol}".split()
    popen = subprocess.Popen(args, stdout=subprocess.PIPE)
    popen.wait()
    output = popen.stdout.read()

In [23]:
all_folders = [f.name for f in os.scandir(SATELLITE_CACHE_FOLDER) if f.is_dir()]

In [24]:
for subfolder in all_folders:
    preproc_data(SATELLITE_CACHE_FOLDER, subfolder)

KeyboardInterrupt: 

In [14]:
class KMPredict(ulog.Loggable):
    def __init__(self, aoi=AOI, log_abbrev="KMP.P"):
        super().__init__(log_abbrev)
        self.cfg = {
            "data_dir": ".SAFE",
            "product": PRODUCT_TYPE,
            "overlapping": 0.0625,
            "tile_size": 512,
            "batch_size": 1,
        }
        self.aoi = AOI
        self.cm_vsm_executable = "cm_vsm"
        self.cm_vsm_env = None
        self.product_name = ""
        self.data_folder = SATELLITE_CACHE_FOLDER
        self.weights_folder = "weights"
        self.predict_folder = OUTPUT_FOLDER
        self.big_image_folder = OUTPUT_FOLDER
        self.weights = ""
        self.product = PRODUCT_TYPE
        self.overlapping = 0.0625
        self.tile_size = 512
        self.resampling_method = "sinc"
        self.features = ["AOT", "B01", "B02", "B03", "B04", "B05", "B06", "B08", "B8A", "B09", "B11", "B12", "WVP"]
        self.classes = [
            "UNDEFINED", "CLEAR", "CLOUD_SHADOW", "SEMI_TRANSPARENT_CLOUD", "CLOUD", "MISSING"
        ]
        self.batch_size = 1
        self.product_safe = ""
        self.product_cvat = ""
        self.weights_path = ""
        self.prediction_product_path = ""
        self.architecture = "Unet"
        self.params = {'path_input': self.product_cvat,
                       'batch_size': self.batch_size,
                       'features': self.features,
                       'dim': self.tile_size,
                       'num_classes': len(self.classes)
                       }
        self.cm_vsm_version = "-"
        self.model = None
        self.aoi_geom = None

    def create_folders(self):
        """
        Create data and weights folders if they do not exist
        """
        if not os.path.exists(self.data_folder):
            os.mkdir(self.data_folder)
        if not os.path.exists(self.weights_folder):
            os.mkdir(self.weights_folder)
        if not os.path.exists(self.predict_folder):
            os.mkdir(self.predict_folder)
        if not os.path.exists(self.big_image_folder):
            os.mkdir(self.big_image_folder)
        if not os.path.exists(self.prediction_product_path):
            os.mkdir(self.prediction_product_path)

    def config_from_dict(self, d, product_name):
        """
        Load configuration from a dictionary.
        :param d: Dictionary with the configuration tree.
        :param product_name: Sentinel-2 product name.
        """
        if "cm_vsm_executable" in d:
            self.cm_vsm_executable = d["cm_vsm_executable"]
        elif "cm_vsm" in d:
            if "path" in d["cm_vsm"]:
                self.cm_vsm_executable = d["cm_vsm"]["path"]
            if "env" in d["cm_vsm"]:
                self.cm_vsm_env = d["cm_vsm"]["env"]

        if product_name:
            self.product_name = product_name
        else:
            self.product_name = d["product_name"]
        
        self.weights = '%s_%s.hdf5' % (self.product.lower(), d["architecture"].lower())
        if self.product == "L2A":
            if d["architecture"] == "DeepLabv3Plus":
                self.features = ["AOT", "B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B11","B12", "WVP"]
            elif d["architecture"] == "Unet":
                self.features = ["AOT", "B01", "B02", "B03", "B04", "B05", "B06", "B08", "B8A", "B09", "B11","B12", "WVP"]
        elif self.product == "L1C":
            self.features = ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B10", "B11", "B12"]  
        self.overlapping = d["overlapping"]
        self.tile_size = d["tile_size"]
        self.resampling_method = d["resampling_method"]
        self.batch_size = d["batch_size"]
        self.architecture = d["architecture"]
        self.data_folder = SATELLITE_CACHE_FOLDER

        self.product_safe = os.path.join(self.data_folder, str(self.product_name + ".SAFE"))
        self.weights_path = os.path.join(self.weights_folder, self.weights)
        self.prediction_product_path = os.path.join(self.predict_folder, self.product_name)

        self.product_cvat = os.path.join(self.data_folder, (self.product_name + ".CVAT"))
        self.aoi_geometry = self.aoi

    def load_config(self, path, product_name):
        with open(path, "rt") as fi:
            self.cfg = json.load(fi)
        self.config_from_dict(self.cfg, product_name)
        self.create_folders()

        overlap_pix = self.overlapping * self.tile_size
        if (overlap_pix % 2) != 0:
            raise Exception('Even number of pixels needed')

    def get_model_by_name(self, name):
        if self.architecture in ARCH_MAP:
            self.model = ARCH_MAP[name]()
            return self.model
        else:
            raise ValueError(("Unsupported architecture \"{}\"."
                              " Only the following architectures are supported: {}.").format(name, ARCH_MAP.keys()))

    def get_cm_vsm_version(self):
        """
        Get the version of the cm-vsm utility.
        """
        q = [self.cm_vsm_executable, "--version"]
        with subprocess.Popen(q, stdout=subprocess.PIPE, env=self.cm_vsm_env) as cm_vsm_process:
            for line in cm_vsm_process.stdout:
                cm_vsm_output = line.decode("utf-8").rstrip("\n")
                if "Version:" in cm_vsm_output:
                    self.cm_vsm_version = cm_vsm_output.split(":")[1]
        return self.cm_vsm_version

    def sub_tile(self, path_out, aoi_geom):
        """
        Execute cm-vsm sub-tiling process
        """
        if aoi_geom is not None:
            self.aoi_geom = aoi_geom

        cm_vsm_query = [
            self.cm_vsm_executable,
            "-j", "-1",
            "-d", os.path.abspath(self.product_safe),
            "-b", ",".join(self.features),
            "-S", str(self.tile_size),
            "-f", "0",
            "-m", self.resampling_method,
            "-o", str(self.overlapping)
        ]
        if path_out and len(path_out) > 0:
            cm_vsm_query += ["-O", path_out]
            self.product_cvat = path_out
        # Area of interest geometry supplied?
        if self.aoi_geom is not None:
            cm_vsm_query += ["-g", self.aoi_geom]

        self.log.info("Splitting with CM-VSM: {}".format(cm_vsm_query))
        with subprocess.Popen(cm_vsm_query, stdout=subprocess.PIPE, env=self.cm_vsm_env) as cm_vsm_process:
            for line in cm_vsm_process.stdout:
                cm_vsm_output = line.decode("utf-8").rstrip("\n")
                self.log.info(cm_vsm_output)
        self.log.info("Sub-tiling has been done!")

    def predict(self, force_predict = False):
        """
        Run prediction for every sub-folder
        """
        # Initialize model
        self.get_model_by_name(self.architecture)

        # Propagate configuration parameters.
        self.model.set_batch_size(self.batch_size)

        # Construct and compile the model.
        self.model.construct(self.tile_size, self.tile_size, len(self.features), len(self.classes))
        self.model.compile()

        # Load model weights.
        self.model.load_weights(self.weights_path)

        # Go through all folders
        date_match = self.product_name.rsplit('_', 1)[-1]
        index_match = self.product_name.rsplit('_', 1)[0].rsplit('_', 1)[-1]

        tile_paths = []

        # Look for .nc file, as the name is not specified
        for subfolder in os.listdir(self.product_cvat):
            subfolder_path = os.path.join(self.product_cvat, subfolder)
            if os.path.isdir(subfolder_path):
                for file in os.listdir(subfolder_path):
                    if file.endswith(".nc"):
                        tile_paths.append(os.path.join(subfolder_path, file))

        # Initialize data generator
        self.params = {'path_input': self.product_cvat,
                       'architecture': self.architecture,
                       'batch_size': self.batch_size,
                       'features': self.features,
                       'tile_size': self.tile_size,
                       'num_classes': len(self.classes),
                       'product_level': self.product,
                       'shuffle': False
                       }

        #Check if prediction already exists
        tile_paths_unseen = []
        for tp in tile_paths:
            path_image = tp.split('/')[-2:-1][0]
            prediction_path = os.path.join(self.prediction_product_path, path_image, 'prediction.png')

            # If True, run prediction on all tiles, else only on those for which prediction.png does not exist 
            if force_predict:
                tile_paths_unseen.append(tp)
            else:
                if not os.path.exists(prediction_path):
                    tile_paths_unseen.append(tp)

        # Predict in batches
        for j in range(0, len(tile_paths_unseen), self.batch_size):
            tile_paths_subset = tile_paths_unseen[j:(j + self.batch_size)]
            self.params['batch_size'] = len(tile_paths_subset)
            predict_generator = DataGenerator(tile_paths_subset, **self.params)

            # Run prediction
            predictions = self.model.predict(predict_generator)
            y_pred = np.argmax(predictions, axis=3)
            for i, prediction in enumerate(predictions):
                save_masks_contrast(tile_paths_subset[i], prediction, y_pred[i], self.prediction_product_path, self.classes)
        return

    def mosaic(self):
        """
        Make a mosaic output from obtained predictions with an overlapping argument
        """
        # Create /prediction/<product_name> directory
        big_image_product = os.path.join(self.big_image_folder, self.product_name)
        if not os.path.exists(big_image_product):
            os.mkdir(big_image_product)

        # Create list of prediction images
        image_list = []
        for subfolder in os.listdir(self.prediction_product_path):
            if os.path.isdir(os.path.join(self.prediction_product_path, subfolder)):
                image_list.append(pathlib.Path(os.path.join(self.prediction_product_path, subfolder, "prediction.png")))

        # Sort images by asc (e.g. 0_0, 0_1, 0_2)
        image_list.sort(key=lambda var: get_img_entry_id(var))

        """
        A function that creates raster mosaic.
        As parameters it takes: list of images, number of tiles per row and number of columns
        
        1) Takes the sub-tile width and height from the first image in the list
        2) Sets final image size from col*width, rows*height
        3) Creates final image from all sub-tiles, bounding box parameters are also set 
        """
        overlap_pix = self.overlapping * self.tile_size
        crop_coef = int(overlap_pix / 2)
        n_rows = math.ceil(10980 / (self.tile_size - crop_coef))
        new_im = image_grid_overlap(image_list, rows=n_rows, cols=n_rows, crop=crop_coef)

        ImageFile.LOAD_TRUNCATED_IMAGES = True
        Image.MAX_IMAGE_PIXELS = None

        # For a correct georeference it is necessary to use 10m resolution band
        jp2 = ''
        if self.product == "L2A":
            for root, dirs, files in os.walk(self.product_safe):
                if root.endswith("R10m"):
                    for file in files:
                        if file.endswith(".jp2"):
                            jp2 = os.path.join(root, file)
        elif self.product == "L1C":
            for root, dirs, files in os.walk(self.product_safe):
                if root.endswith("IMG_DATA"):
                    for file in files:
                        if file.endswith("B02.jp2"):
                            jp2 = os.path.join(root, file)

        # Define a directory where to save a new file, resolution, etc.
        # Get name and index from product name
        date_name = self.product_name.rsplit('_', 4)[0].rsplit('_', 1)[1]
        index_name = self.product_name.rsplit('_', 1)[0].rsplit('_', 1)[-1]

        # Define the output names
        png_name = os.path.join(big_image_product, self.product + "_" + index_name + "_" + date_name + '_KZ_10m.png')
        tif_name = os.path.join(big_image_product, self.product + "_" + index_name + "_" + date_name + '_KZ_10m.tif')

        # Crop the edges in the final image
        f_tile_size = (self.tile_size - crop_coef * 2) * n_rows
        crop = f_tile_size - 10980
        new_im_cropped = ImageOps.crop(new_im, (0, 0, crop, crop))

        # Fill metadata for PNG format
        metadata = PngInfo()
        metadata.add_text("Software", "KM_PREDICT {}; CM_VSM {}".format(__version__, str(self.cm_vsm_version).strip()))

        # Save with a recommended quality and metadata for png, tif is done further down
        new_im_cropped.save(png_name, "PNG", quality=95, pnginfo=metadata)
        new_im_cropped.save(tif_name, "TIFF", quality=95)

        # Deal with tiff-related issues: projection, bands, tags
        proj_rasterio(jp2, tif_name)
        '''
        Assign 0-255 to 0-5 output
        Save final single band raster
        '''

        # Read band 1 (out of 3, they're identical)
        with rasterio.open(tif_name) as tif:
            profile = tif.profile.copy()
            band1 = tif.read(1)

            # Translate values
            band1[band1 == 0] = 0
            band1[band1 == 66] = 1
            band1[band1 == 129] = 2
            band1[band1 == 192] = 3
            band1[band1 == 255] = 4
            band1[band1 == 20] = 5

            profile.update({"count": 1})

            with rasterio.open(tif_name, 'w', **profile) as dst:
                dst.write(band1, 1)

        # Add a version tag for tiff image
        tif_img = Image.open(tif_name)
        tif_img.tag[305] = "KM_PREDICT {}; CM_VSM {}".format(__version__, str(self.cm_vsm_version).strip())
        tif_img.save(tif_name, tiffinfo=tif_img.tag)


In [15]:
def parse_args():
    p = argparse.ArgumentParser()
    p.add_argument("-c", "--config", action="store", default='config/config_example.json', 
                   dest="path_config", help="Path to the configuration file.")
    p.add_argument("-product", "--product", action="store", dest="product_name",
                   help="Optional argument to override product name in config.")
    p.add_argument("-t", "--no-tiling", action="store_true", dest="no_sub_tiling", default=False,
                   help="Disable sub-tiling (the tile output directory has already been created).")
    p.add_argument("-cpu", "--use-cpu", action="store_true", dest="use_cpu", default=False,
                   help="Use CPU.")
    p.add_argument("-f", "--force-predict", action="store_true", dest="force_predict", default=False, help="Force prediction on the tiles for which prediction.png already exists.")
    p.add_argument("-v", "--verbosity", action="store", dest="verbosity", default=1,
                   help="Verbosity level for logging: 0-WARNING, 1-INFO, 2-DEBUG. Default is 1.")
    p.add_argument("-l", "--log-file", action="store", dest="log_file_path",
                   default='km_predict.log',
                   help="Optional argument to specify a location for .log file.")
    p.add_argument("-O", "--tiling-output", action="store", dest="path_out_tiling",
                   help="Override the path to the tiling output directory.")

    args = p.parse_args(args=[])
    return args

In [16]:
def to_crs(poly, target, current='EPSG:4326'):
    project = pyproj.Transformer.from_crs(pyproj.CRS(current), pyproj.CRS(target), always_xy=True).transform
    transformed_poly = transform(project, poly)
    return transformed_poly 

In [17]:
def crop(input_path, output_path, polygon, date, name=None, colormap=None):
    with rasterio.open(input_path) as src:
        out_image, out_transform = rasterio.mask.mask(src, [polygon], crop=True)
        out_meta = src.meta
        
        out_meta.update(driver='GTiff',
                        height=out_image.shape[1],
                        width=out_image.shape[2],
                        transform=out_transform,
                        nodata=0, )

    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.update_tags(start_date=date, end_date=date)
        if name:
            dest.update_tags(name=name)
        if colormap:
            dest.update_tags(colormap=colormap)
        dest.write(out_image)

In [18]:
args = parse_args()
log = ulog.init_logging(int(args.verbosity), "km_predict", "KMP", args.log_file_path)

if args.use_cpu:
    tf.config.set_visible_devices([], 'GPU')

for fold in all_folders:
    kmf = KMPredict(aoi=shapely_polygon)
    kmf.load_config(args.path_config, fold.split(".")[0])

    kmf.sub_tile(args.path_out_tiling, AOI)

    kmf.predict(force_predict = args.force_predict)
    kmf.mosaic()


Using file logger for km_predict.log
INFO: KMP: ===========================================================================


INFO:KMP:===========================================================================


INFO: KMP: ========== km_predict version 1.1.0 


INFO:KMP:========== km_predict version 1.1.0 


INFO: KMP: ===========================================================================


...

terminate called after throwing an instance of 'GDALOGRException'
INFO:KMP.P:Projecting AOI polygon into pixel coordinates.


INFO: KMP.P: Sub-tiling has been done!


  what():  GDAL OGR error : Failed to create a coordinate transformation, No error
Magick: abort due to signal 6 (SIGABRT) "Abort"...
INFO:KMP.P:Sub-tiling has been done!
2023-02-22 13:24:27.616082: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-02-22 13:24:28.706366: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1006] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-22 13:24:28.706488: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 3050 Laptop GPU major: 8 minor: 6 memoryClockRate(GHz): 1.5
pciBusID: 0000:01:00.0
2023-02-22 13:24:28.706563: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory
2023-02-22 

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 512, 512, 13 0                                            
__________________________________________________________________________________________________
custom_pad (CustomPad)          (None, 513, 513, 13) 0           input[0][0]                      
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 32) 3744        custom_pad[0][0]                 
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 32) 128         conv2d[0][0]                     
____________________________________________________________________________________________

FileNotFoundError: [Errno 2] No such file or directory: './img/S2A_MSIL1C_20200728T161911_N0209_R040_T16TFK_20200728T194547.CVAT'

In [ ]:
def transform_crs(data_path, save_path, dst_crs="EPSG:4326", resolution=(10, 10)):
    with rasterio.open(data_path) as src:
        if resolution is None:
            transform, width, height = calculate_default_transform(
                src.crs, dst_crs, src.width, src.height, *src.bounds
            )
        else:
            transform, width, height = calculate_default_transform(
                src.crs,
                dst_crs,
                src.width,
                src.height,
                *src.bounds,
                resolution=resolution,
            )
        kwargs = src.meta.copy()
        kwargs.update(
            {"crs": dst_crs, "transform": transform, "width": width, "height": height}
        )
        with rasterio.open(save_path, "w", **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest,
                )

    return save_path

In [ ]:
def stitch_tiles(paths, out_raster_path, date, name=None, colormap=None):
    if not isinstance(paths[0], str):
        paths = [str(x) for x in paths]
    tiles = []
    tmp_files = []
    
    crs = None
    meta = None
    for i, path in enumerate(paths):
        if i == 0:
            file = rasterio.open(path)
            meta, crs = file.meta, file.crs
        else:
            tmp_path = path.replace(
                '.jp2', '_tmp.jp2').replace('.tif', '_tmp.tif')
            crs_transformed = transform_crs(path, tmp_path, 
                                            dst_crs=crs, 
                                            resolution=None)
            tmp_files.append(crs_transformed)
            file = rasterio.open(crs_transformed)
        tiles.append(file)
            
    tile_arr, transform = merge(tiles, method='last')
    
    meta.update({"driver": "GTiff",
                 "height": tile_arr.shape[1],
                 "width": tile_arr.shape[2],
                 "transform": transform,
                 "crs": crs})
    
    if '.jp2' in out_raster_path:
        out_raster_path = out_raster_path.replace('.jp2', '.tif')
    print(f'saved raster {out_raster_path}')

    for tile in tiles:
        tile.close()
        
    for tmp_file in tmp_files:
        try:
            os.remove(tmp_file)
        except FileNotFoundError:
            print(f'Tile {tmp_file} was removed or renamed, skipping')
        
    with rasterio.open(out_raster_path, "w", **meta) as dst:
        dst.update_tags(start_date=date, end_date=date)
        if name:
            dst.update_tags(name=name)
      
        dst.write(tile_arr)
    
    return out_raster_path

In [ ]:
list_of_pred = os.listdir(OUTPUT_FOLDER)
tiles = []
for i in list_of_pred:
    pt = os.path.join(OUTPUT_FOLDER, i)
    for file in os.listdir(pt):
        if file.endswith(".tif"):
            tiles.append(os.path.join(pt, file))
if len(tiles) > 1:
    stitch_tiles(tiles, 'out.tif', START_DATE)